Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [222]:
import json
import os
from pandas import json_normalize
import psycopg2
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np
from math import floor

load_dotenv()

ACCESS_TOKEN_MCENTER = os.getenv("ACCESS_TOKEN_MCENTER")
ACCESS_TOKEN_BUENOSHOPS = os.getenv("ACCESS_TOKEN_BUENOSHOPS")
ACCESS_TOKEN_MUSICALCRIS = os.getenv("ACCESS_TOKEN_MUSICALCRIS")

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

SMARTGO_TOKEN = os.getenv("SMARTGO_TOKEN")

# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
# variaveis de projeto
empresa = "MCENTER"
table_ful_stock = "mcenter_fulfillment_stock"
table_orders = "mcenter_ml_orders"
table_items = "mcenter_items"
table_tf = "tiny_fulfillment_mcenter"
table_types = "mcenter_types"

In [3]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

In [4]:
def get_wms_data(SMARTGO_TOKEN):
    try:
        url = "https://apigateway.smartgo.com.br/estoque/saldo"

        payload = {}
        headers = {"api_key": SMARTGO_TOKEN}

        response = requests.request("GET", url, headers=headers, data=payload)

        if response.status_code == 200:
            data = json.loads(response.text)

            items = data.get("model", {}).get("items", [])

            result_list = []

            for item in items:
                result_dict = {
                    "id_depositante": item.get("idDepositante"),
                    "depositante": item.get("depositante"),
                    "area": item.get("area"),
                    "areaComputaSaldo": item.get("areaComputaSaldo"),
                    "idProduto": item.get("idProduto"),
                    "produto_nome": item.get("produtoNome"),
                    "produtoCodigoInterno": item.get("produtoCodigoInterno"),
                    "produtoCodigoExterno": item.get("produtoCodigoExterno"),
                    "quantidade": item.get("quantidade"),
                    "quantidadeProduto": item.get("quantidadeProduto"),
                    "quantidadeDeMovimentacao": item.get("quantidadeDeMovimentacao"),
                    "quantidadeProdutosEmbalagem": item.get(
                        "quantidadeProdutosEmbalagem"
                    ),
                    "tipoUnidadeEmbalagem": item.get("tipoUnidadeEmbalagem"),
                    "tipoUnidadeMovimentacao": item.get("tipoUnidadeMovimentacao"),
                    "tipoUnidadeProduto": item.get("tipoUnidadeProduto"),
                    "quantidadeEnderecos": item.get("quantidadeEnderecos"),
                    "quantidade_disponivel": item.get("quantidadeDisponivel"),
                    "quantidadeEmExpedicao": item.get("quantidadeEmExpedicao"),
                    "pedidosCodigosExternos": item.get("pedidosCodigosExternos"),
                    "codigosDeIdentificacao": item.get("codigosDeIdentificacao"),
                    "notasFiscais": item.get("notasFiscais"),
                    "depositos": item.get("depositos"),
                }
                result_list.append(result_dict)

            df = pd.DataFrame(result_list)
            cols = [
                "id_depositante",
                "depositante",
                "idProduto",
                "produto_nome",
                "produtoCodigoInterno",
                "produtoCodigoExterno",
                "quantidade_disponivel",
            ]
            df = df[cols]
            return df

    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
        return None

    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

### Período a consultar

In [5]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 12, 1).date()
data_fim = datetime.now().date()
# data_fim = data_fim + timedelta(days=1)  # + 1 dia para pegar a data atual no DB
# data_fim = data_fim - timedelta(days=1)  # + 1 dia para pegar a data atual no DB
print("data_inicio: ", data_inicio)
print("data_fim: ", data_fim)

data_inicio:  2023-12-01
data_fim:  2024-01-05


### Historico de estoque fulfillment

In [6]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM {table_ful_stock} WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim};'"
    # sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-12-04' AND '2023-12-05';"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '2023-12-01' AND '2024-01-05;'


C:\Users\couti\AppData\Local\Temp\ipykernel_2248\226090857.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [7]:
# df_stock
# print(df_stock.shape)
# duplicates = df_stock[df_stock.duplicated(keep=False)]
# duplicates

In [8]:
# # datas consultadas, dias em que um produto pode ou não estar disponível
# # df_stock["created_at"].value_counts().index.to_list()
# df_stock["created_at"].value_counts().sort_index()
# df_stock["created_at"].dt.date.value_counts().sort_index()
# # df_stock["created_at"].dt.date.value_counts()

# # len(df_stock["created_at"].dt.date.value_counts().sort_index())

In [9]:
# df_stock

In [10]:
# # buscando anuncio e verificando dias consultados
# result = df_stock[df_stock['ml_inventory_id'] == 'FBAY80623']
# result = result.sort_values(by='created_at')
# result

In [11]:
# # encontrando anuncios pela data
# df_stock["created_at"] = pd.to_datetime(df_stock["created_at"])
# result = df_stock.loc[df_stock['created_at'].dt.date == pd.to_datetime('2023-12-7').date()]
# result

In [12]:
# df_stock[df_stock['created_at']=='2023-12-07 03:01:03.036386']

### Ordenando stock por data

In [13]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)

In [14]:
# df_stock

In [15]:
# df_stock.dtypes

In [16]:
# df_stock['data'] = pd.to_datetime(df_stock['data'])

# dfx = condf(df_stock, 'data', '2023-11-1')
# dfx

In [17]:
# # A linha abaixo conta as ocorrências de cada valor em 'ml_inventory_id'
# # value_counts = result['ml_inventory_id'].value_counts()
# value_counts = df_stock['ml_inventory_id'].value_counts()

# # Filtra para incluir apenas aqueles que ocorrem mais de 2 vezes
# filtered_counts = value_counts[value_counts > 1]

# filtered_counts.shape
# # filtered_counts
# value_counts

### Cria coluna has_stock

In [18]:
# df_stock[df_stock["detail_status"] == "transfer"]

In [19]:
# condf(df_stock,'ml_inventory_id','CRMK90073')

In [20]:
# condf(df_stock, "detail_status", "transfer")

In [21]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by="data", ascending=False).reset_index(drop=True)

In [22]:
# df_stock

In [23]:
# condf(df_stock, "ml_inventory_id", "DSGP06967")

In [24]:
# print(df_stock.shape)
# df_stock = df_stock.drop_duplicates()
# print(df_stock.shape)

#### Dias em que produto esteve disponível

In [25]:
# condf(df_stock,'ml_inventory_id','ABCB20467')

In [26]:
## Contando dias em que produto esteve disponível
days_available = df_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
days_available = days_available.rename(columns={"has_stock": "days_available"})

In [27]:
# days_available

In [28]:
# days_available.sort_values(by='days_available',ascending=False)
# days_available['days_available'].value_counts()

# ## Contando dias em que produto esteve disponível
# days_available = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available = days_available.rename(columns={"has_stock": "days_available"})
# days_available

In [29]:
# condf(df_stock, "ml_inventory_id", "MENA66694")

In [30]:
# ## Contando dias em que produto esteve disponível
# days_available_ = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available_ = days_available_.rename(columns={"has_stock": "days_available"})
# days_available_

In [31]:
# condf(days_available, "ml_inventory_id", "DSGP06967")
# condf(days_available, "ml_inventory_id", "DSGP06967")

In [32]:
# Unindo DFs
df_stock_days = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# df_stock_days_left = df_stock.merge(days_available, on=["ml_inventory_id"], how="left")
# df_stock.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# df_stock = df_stock.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# Verificando o formato após a correção
# print(df_stock.shape)
#

In [33]:
# print(df_stock.shape)
# print(df_stock_days.shape)
# print(df_stock_days_left.shape)
# df_stock_days.sample(3)

In [34]:
# # Unindo DFs
# df_stock_ = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# # df_stock_.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# # df_stock_ = df_stock_.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# # Verificando o formato após a correção
# print(df_stock_.shape)
# df_stock_

In [35]:
# data de hoje
data_de_hoje = datetime.now().date()
data_de_hoje = data_de_hoje - timedelta(days=1)
# print(data_de_hoje)

df_stock_days["data"] = pd.to_datetime(df_stock_days["data"])

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock_days[df_stock_days["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)

# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [36]:
df_stock_today

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_available_quantity
0,WQLG77218,5,transfer,1.0,MLB1942106803,nan,2024-01-04,True,2,6
35,NOMF53205,1,None,NaN,MLB1629586589,nan,2024-01-04,True,35,1
70,HGRL10383,2,None,NaN,MLB2003270402,nan,2024-01-04,True,33,2
105,BPGQ10723,12,noFiscalCoverage,12.0,MLB2001140622,nan,2024-01-04,True,24,12
140,KBOK48388,2,None,NaN,MLB1459771919,51818479775,2024-01-04,True,35,2
...,...,...,...,...,...,...,...,...,...,...
12565,CDBO10673,8,None,NaN,MLB2006338750,nan,2024-01-04,True,35,8
12600,NZGA07022,2,None,NaN,MLB2006323551,nan,2024-01-04,True,32,2
12635,PMPX06029,8,None,NaN,MLB2006353434,nan,2024-01-04,True,35,8
12670,VUJI10625,12,transfer,1.0,MLB2006357183,nan,2024-01-04,True,35,13


In [37]:
# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [38]:
# df_stock_today.sample(5)

In [39]:
# condf(df_stock,'ml_inventory_id','FSNB97805')
# condf(df_stock_today,'ml_inventory_id','FSNB97805')

### Buscando hitorico de orders no BD

In [40]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM {table_orders} WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# # Ordenando orders por data
# df_orders = df_orders.sort_values(by="date_approved", ascending=False)
# df_orders["data"] = df_orders["date_approved"].dt.date
# df_orders = df_orders.drop(["date_closed","date_approved"], axis=1)

# filtros
df_orders = df_orders[df_orders["logistic_type"] == "fulfillment"]
# df_orders = df_orders.drop(columns=['category_id','pack_id','variation_attributes_id','variation_name','variation_value_id', 'data'])
df_orders = df_orders.drop(
    columns=[
        "category_id",
        "pack_id",
        "variation_attributes_id",
        "variation_name",
        "variation_value_id",
    ]
)

# change column
df_orders["variation_id"] = df_orders["variation_id"].replace("nan", "0", regex=True)

# df_orders = df_orders[df_orders["order_status"] == "paid"]
# df_orders = df_orders[df_orders["payment_status"] == "approved"]


# df_orders = df_orders.drop(
#     ["pack_id", "date_approved", "order_status", "payment_status"], axis=1
# )
df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '2023-12-01' AND '2024-01-05'


C:\Users\couti\AppData\Local\Temp\ipykernel_2248\3933407791.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)


In [41]:
print(df_orders.shape)
df_orders = df_orders.drop_duplicates()
print(df_orders.shape)

(4104, 13)
(3788, 13)


In [42]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_approved", ascending=False)
df_orders["data"] = df_orders["date_approved"].dt.date
df_orders = df_orders.drop(["date_closed", "date_approved"], axis=1)

In [43]:
df_orders.sample(3)

,ml_code,variation_id,seller_sku,order_id,sold_quantity,title,order_status,payment_status,shipping_id,variation_value_name,logistic_type,data
2130,MLB3461876520,0,FULLSG5145TP,2000007110589820,1,Encordoamento Sg Cordas 009 Para Guitarra - Ki...,paid,approved,42892048081,None,fulfillment,2023-12-10
4835,MLB2192269456,174349843073,FULLMEMPHISHONUSG,2000007258273320,1,Ukulele Soprano Memphis Tagima 21 Havaí Honu ...,paid,approved,42955769034,Verde,fulfillment,2023-12-27
1827,MLB1813392401,77726889180,FULLWINNER11021,2000007096742674,1,Ukulele Winner Soprano 21'' Sapele Laminado Se...,paid,approved,42886162723,Marrom,fulfillment,2023-12-08


In [44]:
# df_orders.head()

In [45]:
# df_orders
# df_orders['pack_id'].value_counts()

# # condf(df_orders,'pack_id','2000005059931419')

In [46]:
# print(df_orders.shape)
# df_orders.head(3)

In [47]:
# condf(df_orders, "ml_code", "MLB1992541482")

#### Total de vendas por ml_code e variation_id

In [48]:
# df_orders.shape

In [49]:
# df_orders

In [50]:
# condf(df_orders,'ml_code','MLB3516894673')

In [51]:
# Total de vendas por ml_code e id de variação
df_orders_quantity = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

In [52]:
df_orders_quantity

,ml_code,variation_id,sold_quantity
0,MLB1363602423,0,13
1,MLB1400355425,0,31
2,MLB1401946009,0,7
3,MLB1403147752,48946752091,1
4,MLB1403763907,0,5
...,...,...,...
279,MLB4202595608,0,44
280,MLB4217370456,0,1
281,MLB4237823132,0,9
282,MLB4297958304,0,4


In [53]:
print(f"Número de ml_code únicos: {len(df_orders_quantity['ml_code'].unique())}")
print(
    f"Número de variation_id únicos: {len(df_orders_quantity['variation_id'].unique())}"
)

Número de ml_code únicos: 229
Número de variation_id únicos: 100


In [54]:
# condf(df_orders_quantity,'ml_code','MLB2075516288')
# df_orders_quantity['ml_code'].value_counts()

In [55]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    df_orders_quantity,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales = df_total_sales.drop(
    columns=["sold_quantity_x", "order_status", "payment_status"]
)

In [56]:
print(f"Total de vendas = {df_total_sales.shape[0]}")

Total de vendas = 3788


In [57]:
# df_orders.shape

In [58]:
# df_total_sales.columns

In [59]:
# df_total_sales.sample()

In [60]:
# # # Exibir todas as linhas e colunas
# # pd.set_option('display.max_rows', None)
# # pd.set_option('display.max_columns', None)

# # Voltar ao normal
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')


# # condf(df_total_sales,'ml_code','MLB3427769549')
# # # x = condf(df_orders,'ml_code','MLB3427769549')

# # # x = x.sort_values(by='data')
# # # x.head(15)
# # x = df_orders[df_orders['ml_code']=='MLB3427769549']
# # x['data'] = pd.to_datetime(x['data'])
# # x[x['data']=='2023-11-22']
# # # x['sold_quantity'].value_counts()

# # # counts = x[x['sold_quantity'] > 1]['sold_quantity'].value_counts()
# # # counts

# # y = x.groupby('data')['sold_quantity'].sum().reset_index()
# # y

# # # # x

In [61]:
# x.dtypes

In [62]:
# df_total_sales['data'].value_counts()

In [63]:
# condf(df_total_sales, "ml_code", "MLB4127982748")
# condf(df_total_sales, "ml_code", "MLB4127982748")

In [64]:
# df_total_sales = df_total_sales.drop(["sold_quantity_x", "shipping_id", "data"], axis=1)
# df_total_sales = df_total_sales.drop_duplicates()

In [65]:
# condf(df_total_sales, "ml_code", "MLB1992541482")

Neste ponto temos o total de itens vendidos de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [66]:
# print(df_total_sales.shape)
# df_total_sales.sample()

In [67]:
# condf(df_total_sales, 'ml_code', 'MLB3461718736')
# condf(df_orders, 'ml_code', 'MLB3461718736')
# # condf(resultado, 'ml_code', 'MLB3427769549')
# # condf(df_stock, 'ml_code', 'MLB3427769549')

In [68]:
# print(df_stock_today.shape)
# df_stock_today.sample()

#### Buscando Produtos

In [69]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)
    sql_query = f"SELECT * FROM {table_items}"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

C:\Users\couti\AppData\Local\Temp\ipykernel_2248\4075562722.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


### separando itens que são catálogos dos que não são

In [70]:
df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]

In [71]:
df_not_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
527,MLB1490210934,SPKV63227,nan,NaN,paused,False,fulfillment


In [72]:
df_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
437,MLB3347847929,WRRA54468,NaN,NaN,paused,True,fulfillment


In [73]:
df_not_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
0,MLB1435852259,TLCG38735,NaN,NaN,active,False,fulfillment
1,MLB1435836214,PUOF38682,NaN,NaN,active,False,fulfillment
2,MLB1400355425,ZXFT21237,NaN,NaN,active,False,fulfillment
3,MLB1440291959,JGRW38703,NaN,NaN,active,False,fulfillment
4,MLB1564998298,OVRP11252,NaN,NaN,paused,False,fulfillment
...,...,...,...,...,...,...,...
527,MLB1490210934,SPKV63227,nan,NaN,paused,False,fulfillment
528,MLB1556975175,DZVJ05686,Marrom,57895096313,active,False,fulfillment
529,MLB950338822,MAEG54107,nan,NaN,paused,False,fulfillment
530,MLB1662970458,UIUW89800,Preto,64498487033,paused,False,fulfillment


In [74]:
print(df_not_catalogo.shape)
print(df_catalogo.shape)

(361, 7)
(171, 7)


In [75]:
df_catalogo

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,logistic_type
19,MLB2170026503,ZNFW24350,NaN,NaN,paused,True,fulfillment
26,MLB3435374559,RYWV54124,NaN,NaN,paused,True,fulfillment
27,MLB3435352761,QYGO52123,NaN,NaN,paused,True,fulfillment
28,MLB3435350955,HGVR43626,NaN,NaN,active,True,fulfillment
29,MLB3435245361,PKNN52962,NaN,NaN,paused,True,fulfillment
...,...,...,...,...,...,...,...
519,MLB4127982748,RLJX42857,nan,NaN,paused,True,fulfillment
520,MLB3497394073,XGSW66673,nan,NaN,paused,True,fulfillment
521,MLB4191526348,LXOO59456,nan,NaN,paused,True,fulfillment
522,MLB3521778415,PSMV02965,nan,NaN,paused,True,fulfillment


In [76]:
# df_orders.sample()

In [77]:
df_total_sales_cat = pd.merge(
    df_catalogo,
    df_total_sales,
    left_on=["ml_code"],
    right_on=["ml_code"],
    how="left",
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_not_cat = pd.merge(
    df_not_catalogo,
    df_total_sales,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="left",
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()

In [78]:
df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].fillna(0)
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].fillna(0)

df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].astype("int64")
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].astype("int64")

In [79]:
# df_total_sales_cat.sample(5)
# df_total_sales_not_cat['total_sold_quantity'].value_counts()
# df_total_sales_cat['total_sold_quantity'].value_counts()

In [80]:
print(df_total_sales_cat.shape)
print(df_total_sales_not_cat.shape)

(2162, 16)
(1512, 15)


In [81]:
# df_total_sales_cat

In [82]:
# df_total_sales_cat = df_total_sales_cat.drop(
#     columns=["data", "shipping_id", "variation_id_x", "order_id"]
# )
df_total_sales_cat_x = df_total_sales_cat.drop(
    columns=["data", "shipping_id", "variation_id_x", "order_id"]
)
df_total_sales_cat_x = df_total_sales_cat_x.drop_duplicates()
df_total_sales_cat_x.shape

(172, 12)

In [83]:
# df_total_sales_cat_x

In [84]:
# df_total_sales_cat['value_name'].value_counts()

In [85]:
# condf(df_total_sales_cat, 'ml_code','MLB3427769549')

In [86]:
# df_vendas_total_sales_not_cat

In [87]:
print(df_total_sales_not_cat.shape)

(1512, 15)


In [88]:
print(df_total_sales_not_cat.shape)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop(
    columns=["data", "shipping_id", "order_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat_x.drop_duplicates()
print(df_total_sales_not_cat_x.shape)

(1512, 15)
(361, 12)


In [89]:
# df_total_sales_cat_x

In [90]:
print(df_total_sales_cat_x.shape)
print(df_total_sales_not_cat_x.shape)

(172, 12)
(361, 12)


In [91]:
# df_total_sales_cat['ml_inventory_id'].value_counts()

In [92]:
# Antes de desabilitar
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# Seu código para exibir o DataFrame

# # Depois de desabilitar
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')


# df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_cat_x = df_total_sales_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)

In [93]:
# df_orders.shape

In [94]:
# condf(df_total_sales_cat,'ml_inventory_id','WTHI04163')
# condf(y,'ml_inventory_id','WTHI04163')

In [95]:
# condf(df_total_sales_cat,'ml_inventory_id','AWAZ99732')

In [96]:
df_total_cat = df_total_sales_cat_x.copy()
df_total_not_cat = df_total_sales_not_cat_x.copy()

In [97]:
df_total_cat.columns

Index(['ml_code', 'ml_inventory_id', 'value_name', 'variation_id_x', 'status',
       'catalog_listing', 'logistic_type_x', 'variation_id_y', 'seller_sku',
       'order_id', 'title', 'shipping_id', 'variation_value_name',
       'logistic_type_y', 'data', 'total_sold_quantity'],
      dtype='object')

In [98]:
# Somando total de vendas por inventory_id
df_sum_qt_sold_cat = (
    df_total_cat.groupby("ml_inventory_id")["total_sold_quantity"].sum().reset_index()
)
df_sum_qt_sold_cat = df_sum_qt_sold_cat.rename(
    columns={"total_sold_quantity": "total_sold_catalog"}
)

df_total_cat = pd.merge(
    df_total_cat,
    df_sum_qt_sold_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)

df_total_cat.rename(columns={"variation_id_y": "variation_id_"})


df_sum_qt_sold_not_cat = (
    df_total_not_cat.groupby("ml_inventory_id")["total_sold_quantity"]
    .sum()
    .reset_index()
)
df_sum_qt_sold_not_cat = df_sum_qt_sold_not_cat.rename(
    columns={"total_sold_quantity": "total_sold_not_catalog"}
)
df_total_not_cat = pd.merge(
    df_total_not_cat,
    df_sum_qt_sold_not_cat[["ml_inventory_id", "total_sold_not_catalog"]],
    on="ml_inventory_id",
    how="left",
)

In [99]:
print(df_total_cat.shape)
print(df_total_not_cat.shape)

(170, 17)
(361, 16)


In [100]:
df_total_cat.rename(columns={"variation_id_y": "variation_id"}, inplace=True)
df_total_cat.shape

(170, 17)

In [101]:
# condf(df_total_cat,'ml_inventory_id','WTHI04163')

In [102]:
df_total_cat = df_total_cat.drop_duplicates(subset=["ml_inventory_id"])

# df_total_cat[df_total_cat['ml_inventory_id']=='WTHI04163']

pergar ml_inventory_id e sum_total e acrescesntar essas informações no df de não catalogo pelo ml_inventory_id
ao fim deixar somatório do catalogo e não catalogo


In [103]:
# cols=['ml_inventory_id','total_sold_catalog']
# df_total_cat = df_total_cat[cols]
# df_total_cat.sample()

In [104]:
df_combined = pd.merge(
    df_total_not_cat,
    df_total_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)
# df_combined.sample()

In [105]:
# df_total_not_cat.shape

In [106]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [107]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [108]:
# df_combined['ml_inventory_id'].value_counts()

In [109]:
print(df_combined.shape)

(361, 17)


In [110]:
df = pd.merge(df_combined, df_stock_today, on="ml_inventory_id", how="left")
df["total_sold_catalog"] = df["total_sold_catalog"].fillna(0).astype("int64")

In [111]:
# df_combined

In [112]:
df.shape

(361, 26)

In [113]:
days = 30

df["total_sold"] = df["total_sold_catalog"] + df["total_sold_not_catalog"]
# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df["period_send_fulfillment"] = np.ceil(
    (df["total_sold"] / df["days_available"]) * days - df["total_available_quantity"]
)

df["period_send_fulfillment"] = df["period_send_fulfillment"].fillna(0)

In [114]:
def calculate_percentual_send(row):
    if row["days_available"] != 0:
        if np.ceil(
            (row["total_sold"] / row["days_available"]) * days * 0.7
            > row["total_available_quantity"]
        ):
            # return (np.ceil(row["total_sold"] / row["days_available"]) * days - row["total_available_quantity"])
            return np.ceil(
                (row["total_sold"] / row["days_available"]) * days
                - row["total_available_quantity"]
            )

    return 0

In [115]:
# Aplicando a função à coluna "percentual_send"
df["stock_replenishment"] = df.apply(calculate_percentual_send, axis=1)

In [116]:
# df.sample(5)

In [117]:
# df.columns

In [118]:
# df.shape

In [119]:
# df

In [120]:
df_have_itens = df[df["days_available"] > 0]
# df_have_itens.shape
# df_have_itens

In [121]:
# produtos sem estoque no período
df_no_itens = df[df["days_available"] <= 0]
# df_no_itens = df_no_itens.drop(columns=["period_send_fulfillment"])
# df_no_itens.shape

In [122]:
df_sold_zero = df_have_itens[df_have_itens["total_sold"] == 0]
df_sold = df_have_itens[df_have_itens["total_sold"] > 0]

In [123]:
# print(df_no_itens.shape)
# print(df_sold.shape)
# print(df_sold_zero.shape)

In [124]:
# df_sold_zero.sample(3)

In [125]:
# df_sold.sample(3)

In [126]:
dfx = df_have_itens.copy()

In [127]:
# df_sold_zero.columns

In [128]:
# dfx.columns

In [129]:
cols = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "total_available_quantity",
    "days_available",
    "total_sold_not_catalog",
    "total_sold_catalog",
    "total_sold",
    "period_send_fulfillment",
    "stock_replenishment",
]

df_sold_zero = df_sold_zero[cols]
df_sold = df_sold[cols]
df_no_itens = df_no_itens[cols]

In [130]:
def rename_columns(df):
    return df.rename(
        columns={
            "detail_status": "transfer_status",
            "detail_quantity": "transfer_quantity",
        }
    )


df_sold_zero = rename_columns(df_sold_zero)
df_sold = rename_columns(df_sold)
df_no_itens = rename_columns(df_no_itens)

In [131]:
# df_no_itens.columns

In [132]:
# dfx = dfx.rename(
#     columns={"detail_status": "transfer_status", "detail_quantity": "transfer_quantity"}
# )

In [133]:
# dfx.sample()
# dfx.shape

In [134]:
# dfx

### Agrupando por categoria de produto para enviar

In [135]:
df_itens_to_send = df_sold[df_sold["stock_replenishment"] > 0]
df_itens_to_send = df_itens_to_send[["ml_inventory_id", "stock_replenishment"]]
df_itens_to_send = df_itens_to_send.rename(columns={"ml_inventory_id": "inventory_id"})

df_itens_to_send.shape

(66, 2)

In [136]:
# df_itens_to_send.sample()

In [137]:
# Buscando relação fulfillment X tiny
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM {table_tf}"
    print(sql_query)
    df_tiny_fulfillment = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM tiny_fulfillment_mcenter


C:\Users\couti\AppData\Local\Temp\ipykernel_2248\2657413349.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tiny_fulfillment = pd.read_sql(sql_query, conn)


In [138]:
# Buscando categorias dos produtos
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM {table_types}"
    print(sql_query)
    df_types = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM mcenter_types


C:\Users\couti\AppData\Local\Temp\ipykernel_2248\4136613845.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_types = pd.read_sql(sql_query, conn)


In [139]:
df_types.shape

(361, 2)

In [140]:
# Agrupando itens a enviar com tipo de itens

df_send_types = pd.merge(df_itens_to_send, df_types, on="inventory_id", how="inner")

In [141]:
print(df_send_types.shape)
print(df_itens_to_send.shape)

(66, 3)
(66, 2)


In [142]:
df_send_types

,inventory_id,stock_replenishment,type
0,MENA20785,12.0,Percussão
1,GLMK21133,5.0,Pandeiro
2,FXMN50419,70.0,Cabo
3,KRJP31426,4.0,Suporte Sopro
4,HFNJ52229,372.0,Cabo
...,...,...,...
61,CIKE94068,6.0,Cabo rolo
62,RLJX42857,114.0,Pandeiro
63,DZVJ05686,20.0,Ukulele
64,MAEG54107,32.0,Cabo


In [143]:
### Contando estoque da WMS em Produtos sem estoque no período
df_wms = get_wms_data(SMARTGO_TOKEN)

In [227]:
# Identificar todos os tipos únicos
unique_types = df_send_types["type"].unique()

# Criar grupos onde cada grupo contém todas as instâncias associadas a um tipo
result_dfs = []

for unique_type in unique_types:
    type_group = df_send_types[df_send_types["type"] == unique_type]
    result_dfs.append(type_group)

# Exibir os DataFrames resultantes
for i, result_df in enumerate(result_dfs):
    print(f"Grupo {i + 1}:\n{result_df}\n")
#     result_df

Grupo 1:
   inventory_id  stock_replenishment       type
0     MENA20785                 12.0  Percussão
9     TXWS20511                 17.0  Percussão
10    STLG62905                  8.0  Percussão
17    CMXV62869                  7.0  Percussão

Grupo 2:
   inventory_id  stock_replenishment      type
1     GLMK21133                  5.0  Pandeiro
5     LKXA87678                  6.0  Pandeiro
8     CRMK90073                 38.0  Pandeiro
24    MAEG89052                 17.0  Pandeiro
45    WOJB37093                 18.0  Pandeiro
54    RKDM61361                 23.0  Pandeiro
55    RXXA58942                 95.0  Pandeiro
56    PPWL59058                 84.0  Pandeiro
58    QOVS42512                135.0  Pandeiro
62    RLJX42857                114.0  Pandeiro
65    UIUW89800               1843.0  Pandeiro

Grupo 3:
   inventory_id  stock_replenishment  type
2     FXMN50419                 70.0  Cabo
4     HFNJ52229                372.0  Cabo
19    ZOQD52873                 62.0  

In [228]:
# df_wms

In [229]:
# Lista para armazenar os DataFrames resultantes de cada agrupamento
result_dfs_list = []

while any(result_df.shape[0] > 0 for result_df in result_dfs):
    first_rows_dfs = []
    remaining_rows_dfs = []

    for result_df in result_dfs:
        if not result_df.empty:
            # Pega a primeira linha do DataFrame
            first_row_df = result_df.head(1)
            first_rows_dfs.append(first_row_df)

            # Pega as linhas restantes do DataFrame
            remaining_rows_df = result_df.iloc[1:]
            if not remaining_rows_df.empty:
                remaining_rows_dfs.append(remaining_rows_df)
            else:
                # print(
                #     f"DataFrame vazio encontrado após extrair a primeira linha:\n{result_df}"
                # )
                pass

    # Adiciona o DataFrame resultante de cada agrupamento à lista
    result_dfs_list.append(pd.concat(first_rows_dfs, ignore_index=True))

    # Atualiza a lista result_dfs com os DataFrames restantes
    result_dfs = remaining_rows_dfs.copy()

# cols = ['inventory_id', 'ml_code', 'seller_sku', 'title', 'tiny_id', 'tiny_sku', 'stock_replenishment', 'qtd_to_send', 'type']
cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "stock_replenishment",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "qtd_to_send",
    "type",
]

# Exibir os DataFrames resultantes de cada agrupamento
for i, result_df in enumerate(result_dfs_list):
    result_df = pd.merge(result_df, df_tiny_fulfillment, on="inventory_id", how="inner")
    result_df = result_df.drop(["Tipo de produto"], axis=1)
    result_df = result_df.rename(
        columns={
            "Quantidade do item": "qtd_item",
            "SKU": "seller_sku",
            "Título do anúncio": "title",
            "ID Tiny": "tiny_id",
            "SKU Tiny": "tiny_sku",
        }
    )

    result_df["qtd_to_send"] = result_df["stock_replenishment"] * result_df["qtd_item"]
    result_df["qtd_item"] = result_df["qtd_item"].astype("int64")
    result_df["qtd_to_send"] = result_df["qtd_to_send"].astype("int64")

    result_df = result_df[cols]
    ##############################################################################################
    resultado = pd.merge(
        result_df,
        df_wms,
        left_on="tiny_sku",
        right_on="produtoCodigoInterno",
        how="inner",
    )
    resultado["produtoCodigoInterno"].fillna(resultado["tiny_sku"], inplace=True)

    df_envio = resultado.copy()

    # Função para aplicar nas linhas do DataFrame
    def ajustar_envio(row):
        if (row["quantidade_disponivel"] < row["qtd_to_send"]) and (
            row["quantidade_disponivel"] > 0
        ):
            return row["quantidade_disponivel"]
        elif row["quantidade_disponivel"] > row["qtd_to_send"]:
            return row["qtd_to_send"]
        else:
            return 0

    # Criar a coluna 'envio_ajust' aplicando a função nas linhas
    df_envio["envio_ajust"] = df_envio.apply(ajustar_envio, axis=1)

    # Produtos sem estoque
    df_no_stock = df_envio[(df_envio["quantidade_disponivel"] < 0)]

    # Produtos com estoque menor que o solicitado e quantidade disponível maior que 0
    df_less_stock = df_envio[
        (df_envio["quantidade_disponivel"] < df_envio["qtd_to_send"])
        & (df_envio["quantidade_disponivel"] > 0)
    ]

    # Encontrar as linhas comuns entre df_envio e df_no_stock
    common_rows = pd.merge(df_envio, df_no_stock, how="inner")

    # Remover as linhas comuns de df_envio
    df_envio = df_envio[~df_envio.isin(common_rows.to_dict(orient="list")).all(axis=1)]

    # Encontrar as linhas duplicadas com base na coluna 'inventory_id'
    duplicated_rows = df_envio[df_envio.duplicated(subset="inventory_id", keep=False)]

    # Verificar se duplicated_rows não é vazio
    if not duplicated_rows.empty:
        # Criar um novo DataFrame com as linhas duplicadas
        df_duplicatas = duplicated_rows.copy()

        # Agrupar por 'inventory_id'
        grouped_df = df_duplicatas.groupby("inventory_id")

        # Iterar sobre os grupos
        for group_name, group_df in grouped_df:
            # Verificar a condição em cada linha do grupo
            condition_check = (
                group_df["envio_ajust"] > group_df["quantidade_disponivel"]
            )

            # Exibir as linhas que satisfazem a condição
            df_satisfaz_condicao = group_df[condition_check]

            # Calcular a razão e adicionar uma nova coluna
            df_satisfaz_condicao["razao"] = (
                df_satisfaz_condicao["quantidade_disponivel"]
                / df_satisfaz_condicao["qtd_item"]
            )
            df_satisfaz_condicao["razao"] = df_satisfaz_condicao["razao"].apply(floor)

            # Aplicar o valor da razão em df_envio com base na condição
            df_envio.loc[
                df_envio["inventory_id"].isin(df_satisfaz_condicao["inventory_id"]),
                "razao",
            ] = df_satisfaz_condicao["razao"].values

            # Atualizar a coluna 'envio_ajust' com base na nova lógica
            df_envio["envio_ajust"] = df_envio.apply(
                lambda row: row["qtd_item"] * row["razao"]
                if row["razao"] > 1
                else row["envio_ajust"],
                axis=1,
            )

            # Preencher valores nulos na coluna 'razao' em df_envio com 1
            df_envio["razao"] = df_envio["razao"].fillna(1).astype("int64")

    ##############################################################################################
    print(f"Grupo {i + 1}:\n", df_envio)

Grupo 1:
    inventory_id        ml_code          seller_sku  \
0     MENA20785  MLB1400763583        FULLIZZO6342   
1     HSDO10298  MLB2004848109     FULLBASSOVTSL60   
2     PROW53057  MLB1461271134  FULLHERCULESGS412B   
3     JJMG49643  MLB1464478537     FULLKALANI15416   
4     MNSV53924  MLB1629577482            FULLSC30   
5     MNSV53924  MLB1629577482            FULLSC30   
6     TPDH38567  MLB1435969309      FULLELIXIR5338   
7     TPDH38567  MLB1435969309      FULLELIXIR5338   
8     DSGP06979  MLB1992567302           FULLNI7AM   
9     TAPQ85749  MLB2606294854  FULLVANDOREN2CR103   
10    TAPQ85749  MLB2606294854  FULLVANDOREN2CR103   

                                                title  stock_replenishment  \
0   Cuica Profissiona Izzo 08 Pol X 30 Cm Aco Inox...                 12.0   
1   Correia Guitarra Violão Baixo Basso Alça Couro...                 29.0   
2   Suporte Profissional Guitarra Baixo Violão Her...                 29.0   
3   Ukulele Tenor 26 Tribes N

In [206]:
resultado

,inventory_id,ml_code,seller_sku,title,stock_replenishment,tiny_id,tiny_sku,qtd_item,qtd_to_send,type,id_depositante,depositante,idProduto,produto_nome,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
0,LXQV54406,MLB2777157690,FULLERNIEBALLP04052,Correia P/ Violão Guitarra Polypro Marrom P040...,5.0,745431564,749699140528,1,5,Correia,4805,002 - Musical Center Ponta Grossa - Ltda - Matriz,1117662,CORREIA POLYPRO MARROM P04052 ERNIE BALL 12906,749699140528,749699140528,8
1,LXQV54406,MLB2777157690,FULLERNIEBALLP04052,Correia P/ Violão Guitarra Polypro Marrom P040...,5.0,745431564,749699140528,1,5,Correia,4806,003 - Musical Center Ponta Grossa - Ltda - Filial,1057149,CORREIA POLYPRO MARROM P04052 ERNIE BALL 12906,749699140528,12906,20


In [148]:
result_dfs_list[0]

,inventory_id,stock_replenishment,type
0,MENA20785,12.0,Percussão
1,GLMK21133,5.0,Pandeiro
2,FXMN50419,70.0,Cabo
3,KRJP31426,4.0,Suporte Sopro
4,HSDO10298,29.0,Correia
5,PROW53057,29.0,Suporte Corda
6,WOXA67988,42.0,Escaleta
7,JJMG49643,8.0,Ukulele
8,MNSV53924,14.0,Cabo rolo
9,TPDH38567,27.0,Encordoamento


In [149]:
result_dfs_list

[   inventory_id  stock_replenishment           type
 0     MENA20785                 12.0      Percussão
 1     GLMK21133                  5.0       Pandeiro
 2     FXMN50419                 70.0           Cabo
 3     KRJP31426                  4.0  Suporte Sopro
 4     HSDO10298                 29.0        Correia
 5     PROW53057                 29.0  Suporte Corda
 6     WOXA67988                 42.0       Escaleta
 7     JJMG49643                  8.0        Ukulele
 8     MNSV53924                 14.0      Cabo rolo
 9     TPDH38567                 27.0  Encordoamento
 10    DSGP06979                 14.0        Baqueta
 11    TAPQ85749                  6.0        Palheta,
    inventory_id  stock_replenishment           type
 0     TXWS20511                 17.0      Percussão
 1     LKXA87678                  6.0       Pandeiro
 2     HFNJ52229                372.0           Cabo
 3     CJBY66878                  5.0  Suporte Sopro
 4     ESVG06709                  4.0        

In [150]:
result_df

,inventory_id,ml_code,seller_sku,title,stock_replenishment,tiny_id,tiny_sku,qtd_item,qtd_to_send,type
0,LXQV54406,MLB2777157690,FULLERNIEBALLP04052,Correia P/ Violão Guitarra Polypro Marrom P040...,5.0,745431564,749699140528,1,5,Correia


### Contando estoque da WMS em Produtos sem estoque no período

In [151]:
df_wms = get_wms_data(SMARTGO_TOKEN)

In [152]:
df_wms

,id_depositante,depositante,idProduto,produto_nome,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
0,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056192,ALAVANCA DE GUITARRA FLOYD C/ROSCA CROMO (6 UN...,7899548645179,103CR,5
1,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056203,BAG PHX PERSONALIZADO P/ CONJ. PRATO E BAQUETA...,7899548630373,PAA130,77
2,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056204,Bateria Infantil Profissional Para Criança Tur...,7898475680628,TURBINHO-JZ,1
3,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056205,BATERIA PHX DISNEY INFANTIL PRINCESAS MOSAICO ...,7899548641836,BID-P1,2
4,4804,001 - Cristiano Padilha Instrumentos Musicais ...,1056206,"Bateria PHX Infantil 16"" Preta DS516C BK",7899548605579,DS516C BK,4
...,...,...,...,...,...,...,...
1709,4808,005 - Rogerio de Souza Instrumentos Musicais LTDA,1056927,SERRA TICO-TICO BI-METAL STARRETT 75MM 6D BU36,7891265687180,BU36,47
1710,4808,005 - Rogerio de Souza Instrumentos Musicais LTDA,1056815,ENCORDOAMENTO VIOLÃO AÇO MARTIN PH-BRONZE 92/8...,7899153070878,41Y18MA535,9
1711,4808,005 - Rogerio de Souza Instrumentos Musicais LTDA,1115019,PANDEIRO 10 POL ABS BRANCO 6 AFIN PLAT INOX PE...,7908334113003,7908334113003,3
1712,5094,007 - LIVERPOOL INSTRUMENTOS E ACESSORIOS MUSI...,1114387,NIRVANA ECO BIO 7A MADEIRA,NI 7AM,7897937421007,1000


In [153]:
dic_column_name = {"ml_inventory_id": "inventory_id"}
df_no_itens = df_no_itens.rename(columns=dic_column_name)
df_sold_zero = df_sold_zero.rename(columns=dic_column_name)

In [154]:
df_tiny_fulfillment.sample()

,inventory_id,ml_code,SKU,Título do anúncio,ID Tiny,SKU Tiny,Quantidade do item,Tipo de produto
186,QCVC39144,MLB2003129630,7898529532484,Correia Guitarra Violão Baixo Basso Sf 106 Pei...,691339657,PALHETA2,1,Correia


In [155]:
df_tiny_fulfillment_no_itens = pd.merge(
    df_no_itens, df_tiny_fulfillment, on="inventory_id", how="inner"
)

In [156]:
print(df_no_itens.shape)
print(df_tiny_fulfillment.shape)
print(df_tiny_fulfillment_no_itens.shape)

(71, 16)
(499, 8)
(121, 23)


In [157]:
df_tiny_fulfillment_no_itens.columns

Index(['ml_code_x', 'seller_sku', 'inventory_id', 'value_name', 'status',
       'title', 'available_quantity_today', 'transfer_status',
       'transfer_quantity', 'total_available_quantity', 'days_available',
       'total_sold_not_catalog', 'total_sold_catalog', 'total_sold',
       'period_send_fulfillment', 'stock_replenishment', 'ml_code_y', 'SKU',
       'Título do anúncio', 'ID Tiny', 'SKU Tiny', 'Quantidade do item',
       'Tipo de produto'],
      dtype='object')

In [158]:
# df_tiny_fulfillment_no_itens[['ml_code_x','ml_code_y']]
# df_tiny_fulfillment_no_itens[['ml_code_x','seller_sku','SKU', 'Título do anúncio']]

In [159]:
df_tiny_fulfillment_no_itens = df_tiny_fulfillment_no_itens.drop(
    columns=["ml_code_y", "seller_sku", "title"]
)

In [160]:
dic_new_names = {
    "ml_code_x": "ml_code",
    "SKU": "seller_sku",
    "Título do anúncio": "title",
    "Quantidade do item": "qtd_item",
    "ID Tiny": "tiny_id",
    "SKU Tiny": "tiny_sku",
    "Tipo de produto": "type",
}
df_tiny_fulfillment_no_itens = df_tiny_fulfillment_no_itens.rename(
    columns=dic_new_names
)

cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "type",
    "stock_replenishment",
    "status",
]

In [161]:
df_tiny_fulfillment_no_itens.sample()

,ml_code,inventory_id,value_name,status,available_quantity_today,transfer_status,transfer_quantity,total_available_quantity,days_available,total_sold_not_catalog,total_sold_catalog,total_sold,period_send_fulfillment,stock_replenishment,seller_sku,title,tiny_id,tiny_sku,qtd_item,type
115,MLB951043222,IXGP53982,nan,paused,0,None,NaN,0,0,0,2,2,inf,0.0,FULLP10LSHOGUN20FT,Cabo Santo Angelo Shogun L Ouro P10 X P10l - ...,599250891,7899028813135,1,Cabo


In [162]:
df_tiny_fulfillment_no_itens = df_tiny_fulfillment_no_itens[cols]
df_tiny_fulfillment_no_itens

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status
0,OVRP11252,MLB1564998298,FULLDATALINKMICECOC,Cabo Microfone Balanceado Estereo Dmx Rolo Com...,754085190,7898571083767,1,Cabo rolo,0.0,paused
1,JJMG39331,MLB1440293157,FULLWINNER11014,Ukulele Soprano 21 Abs Tampo Sapele Vermelho ...,687918940,7899574332890,1,Ukulele,0.0,paused
2,LKXA44509,MLB1618249173,FULLTIMBRA8629C,Pandeiro 10 Profissional Aro Dourado Pele Pr...,616465636,7897626886292,1,Pandeiro,0.0,paused
3,SSGO43954,MLB1488916154,FULLTIMBRA8291C,Pandeiro 11 Profissional Prat Duplas Pele Pri...,616472756,7897626882911,1,Pandeiro,0.0,paused
4,ERIM51807,MLB949788598,P10LNINJA10FT,Cabo Santo Angelo Ninja L P10 X P10l - 10ft/3.05m,598796832,7899028824407,1,Cabo,0.0,paused
...,...,...,...,...,...,...,...,...,...,...
116,QTPB03410,MLB1992512780,None,Encordoamento 0.11 P/ Violão Aço 85/15 Bronze ...,730228363,7897626866874,1,Encordoamento,0.0,paused
117,VXCN97101,MLB1992506451,None,Encordoamento 0.11 P/ Violão Aço 85/15 Bronze ...,730228363,7897626866874,1,Encordoamento,0.0,paused
118,ITSG97062,MLB1990145508,None,Console Boxx U6 De Mistura 110v/220v,730222570,BOXXU6,1,Mesa,0.0,paused
119,ITDN89121,MLB1965264557,None,Microfone Lapela Tag TG-88LP,602491828,7898651941550,1,Microfone,0.0,paused


In [163]:
df_tiny_fulfillment_no_itens[df_tiny_fulfillment_no_itens["tiny_id"] == "1056269"]

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status


In [164]:
df_wms.sample(2)

,id_depositante,depositante,idProduto,produto_nome,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
1364,4808,005 - Rogerio de Souza Instrumentos Musicais LTDA,1056741,CADERNO ESPIRAL DE MUSICA PQ 100 PAGINAS IZZO 240,7897626802407,240,25
788,4806,003 - Musical Center Ponta Grossa - Ltda - Filial,1057195,MINI PANDEIRO MEIA LUA KIDZZO 17706,7897626807105,17706,20


In [165]:
df_wms_tf_no_itens_ci = pd.merge(
    df_tiny_fulfillment_no_itens,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoInterno",
    how="left",
)
df_wms_tf_no_itens_ce = pd.merge(
    df_tiny_fulfillment_no_itens,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoExterno",
    how="left",
)

In [166]:
print(df_wms_tf_no_itens_ci.shape)
print(df_wms_tf_no_itens_ce.shape)

(255, 17)
(125, 17)


In [167]:
df_wms_tf_no_itens_ci.sample()

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status,id_depositante,depositante,idProduto,produto_nome,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
143,VAJM34569,MLB2001139731,7891088010141,Correia Guitarra Violão Baixo Basso Cristo Sal...,691339657,PALHETA2,1,Correia,0.0,paused,4807.0,004 - M. A. Bueno Frederico Sonorizacao Ltda,1056621.0,Palheta,PALHETA2,None,1270.0


In [168]:
# df_junto = pd.merge(df_wms_tf_no_itens_ci, df_wms_tf_no_itens_ce, on='coluna_comum', how='outer')

df_wms_tf_no_itens = pd.concat(
    [df_wms_tf_no_itens_ci, df_wms_tf_no_itens_ce], ignore_index=True
)

print(df_wms_tf_no_itens.shape)

(380, 17)


In [169]:
df_wms_tf_no_itens = df_wms_tf_no_itens.drop_duplicates()

In [170]:
# df_wms_tf_no_itens.columns

In [171]:
print(df_wms_tf_no_itens.shape)

cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]

df_wms_tf_no_itens = df_wms_tf_no_itens[cols]

df_wms_tf_no_itens["quantidade_disponivel"] = (
    df_wms_tf_no_itens["quantidade_disponivel"].fillna(0).astype("int64")
)

(319, 17)


In [172]:
# Dados onde quantidade_disponivel = 0
df_wms_tf_no_itens_less_zero = df_wms_tf_no_itens[
    df_wms_tf_no_itens["quantidade_disponivel"] < 1
]
df_wms_tf_no_itens_less_zero = df_wms_tf_no_itens_less_zero.drop_duplicates()

print(df_wms_tf_no_itens_less_zero.shape)

(101, 12)


In [173]:
## Removendo e somando duplicatas
# Lista das colunas que devem ser usadas para identificar linhas repetidas
cols_to_check_duplicates = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
]

In [174]:
# Agrupar por linhas repetidas e somar a coluna quantidade_disponivel
df_wms_tf_no_itens_sum = (
    df_wms_tf_no_itens.groupby(cols_to_check_duplicates)["quantidade_disponivel"]
    .sum()
    .reset_index()
)

# organizando
cols_result = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]
df_wms_tf_no_itens_sum = df_wms_tf_no_itens_sum[cols_result]

In [175]:
print(df_wms_tf_no_itens_sum.shape)
print(df_wms_tf_no_itens_less_zero.shape)

(63, 12)
(101, 12)


In [176]:
# Concatene os dois DataFrames verticalmente
df_wms_tf_no_itens = pd.concat(
    [df_wms_tf_no_itens_sum, df_wms_tf_no_itens_less_zero], ignore_index=True
)
# print(df_wms_tf_no_itens.shape)

df_wms_tf_no_itens = df_wms_tf_no_itens.drop_duplicates()
# print(df_wms_tf_no_itens.shape)

In [177]:
# print(df_wms_tf_no_itens.shape)
# print(df_df_wms_tf_no_itens_sum.shape)

In [178]:
# df_wms_tf_no_itens[df_wms_tf_no_itens['inventory_id']=='ELKB12099']
# df_wms_tf_no_itens[df_wms_tf_no_itens['tiny_id']=='754085190']
# df_wms_tf_no_itens[df_wms_tf_no_itens['tiny_sku']=='7898571083767']

In [179]:
# df_df_wms_tf_no_itens_sum[df_df_wms_tf_no_itens_sum['inventory_id']=='ELKB12099']
# df_df_wms_tf_no_itens_sum[df_df_wms_tf_no_itens_sum['tiny_id']=='754085190']
# df_df_wms_tf_no_itens_sum[df_df_wms_tf_no_itens_sum['tiny_sku']=='7898571083767']

### Contando estoque da WMS em Produtos sem vendas no período

In [180]:
df_tiny_fulfillment_sold_zero = pd.merge(
    df_sold_zero, df_tiny_fulfillment, on="inventory_id", how="inner"
)

In [181]:
df_tiny_fulfillment_sold_zero.sample()

,ml_code_x,seller_sku,inventory_id,value_name,status,title,available_quantity_today,transfer_status,transfer_quantity,total_available_quantity,...,total_sold,period_send_fulfillment,stock_replenishment,ml_code_y,SKU,Título do anúncio,ID Tiny,SKU Tiny,Quantidade do item,Tipo de produto
119,MLB2650708267,NaN,VOFT09621,NaN,active,NaN,3,None,NaN,3,...,0,-3.0,0.0,MLB2650708267,FULLC411027,Cordas Violão 011 Aço Custom Light Bronze Nano...,632364112,7897626832114,1,Encordoamento


In [182]:
df_tiny_fulfillment_sold_zero.columns

Index(['ml_code_x', 'seller_sku', 'inventory_id', 'value_name', 'status',
       'title', 'available_quantity_today', 'transfer_status',
       'transfer_quantity', 'total_available_quantity', 'days_available',
       'total_sold_not_catalog', 'total_sold_catalog', 'total_sold',
       'period_send_fulfillment', 'stock_replenishment', 'ml_code_y', 'SKU',
       'Título do anúncio', 'ID Tiny', 'SKU Tiny', 'Quantidade do item',
       'Tipo de produto'],
      dtype='object')

In [183]:
# df_tiny_fulfillment_sold_zero[['ml_code_x','ml_code_y']]
# df_tiny_fulfillment_sold_zero[['ml_code_x','seller_sku','SKU', 'Título do anúncio']]

In [184]:
df_tiny_fulfillment_sold_zero = df_tiny_fulfillment_sold_zero.drop(
    columns=["ml_code_y", "seller_sku", "title"]
)

In [185]:
dic_new_names = {
    "ml_code_x": "ml_code",
    "SKU": "seller_sku",
    "Título do anúncio": "title",
    "Quantidade do item": "qtd_item",
    "ID Tiny": "tiny_id",
    "SKU Tiny": "tiny_sku",
    "Tipo de produto": "type",
}
df_tiny_fulfillment_sold_zero = df_tiny_fulfillment_sold_zero.rename(
    columns=dic_new_names
)

cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "type",
    "stock_replenishment",
    "status",
]

In [186]:
df_tiny_fulfillment_sold_zero.sample()

,ml_code,inventory_id,value_name,status,available_quantity_today,transfer_status,transfer_quantity,total_available_quantity,days_available,total_sold_not_catalog,total_sold_catalog,total_sold,period_send_fulfillment,stock_replenishment,seller_sku,title,tiny_id,tiny_sku,qtd_item,type
51,MLB1847067851,UCDR32919,NaN,active,8,None,NaN,8,35,0,0,0,-8.0,0.0,FULLSG6686011,Encordoamento 0.11 P/ Violão Aço Bronze Custom...,723638523,7897626866867,1,Encordoamento


In [187]:
df_tiny_fulfillment_sold_zero = df_tiny_fulfillment_sold_zero[cols]

In [188]:
df_tiny_fulfillment_sold_zero.sample()

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,type,stock_replenishment,status
21,TNXB83665,MLB1398114866,FULLELIXIR12152,Encordoamento Guitarra Elixir Nanoweb Original...,742019381,7897626857230,1,Encordoamento,0.0,active


In [189]:
df_wms_tf_sold_zero_ci = pd.merge(
    df_tiny_fulfillment_sold_zero,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoInterno",
    how="left",
)
df_wms_tf_sold_zero_ce = pd.merge(
    df_tiny_fulfillment_sold_zero,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoExterno",
    how="left",
)

In [190]:
x = pd.merge(
    df_tiny_fulfillment_sold_zero,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoInterno",
    # how='left'
)
y = pd.merge(
    df_tiny_fulfillment_sold_zero,
    df_wms,
    left_on="tiny_sku",
    right_on="produtoCodigoExterno",
    # how='left'
)

In [191]:
print(df_wms_tf_sold_zero_ci.shape)
print(x.shape)
print(df_wms_tf_sold_zero_ce.shape)
print(y.shape)

(346, 17)
(333, 17)
(168, 17)
(75, 17)


In [192]:
# df_junto = pd.merge(df_wms_tf_no_itens_ci, df_wms_tf_no_itens_ce, on='coluna_comum', how='outer')

df_wms_tf_sold_zero = pd.concat(
    [df_wms_tf_sold_zero_ci, df_wms_tf_sold_zero_ce], ignore_index=True
)

print(df_wms_tf_sold_zero.shape)

(514, 17)


In [193]:
df_wms_tf_sold_zero = df_wms_tf_sold_zero.drop_duplicates()
print(df_wms_tf_sold_zero.shape)

(431, 17)


In [194]:
cols = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]

df_wms_tf_sold_zero = df_wms_tf_sold_zero[cols]
# df_wms_tf_sold_zero = df_wms_tf_sold_zero['quantidade_disponivel'].astype('int64').fillna(0)
df_wms_tf_sold_zero["quantidade_disponivel"] = (
    df_wms_tf_sold_zero["quantidade_disponivel"].fillna(0).astype("int64")
)

df_wms_tf_sold_zero.sample()

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,stock_replenishment,status,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
71,ELKX63703,MLB1490289695,FULLIZZO6232,Bacurinha Izzo 8 X 30 Cm Aluminio Pele Leitos...,680027823,7897626862326,1,0.0,active,7897626862326,7897626862326,13


In [195]:
df_wms_tf_sold_zero[df_wms_tf_sold_zero["inventory_id"] == "YAZB38863"]
# df_wms_tf_sold_zero[df_wms_tf_sold_zero["tiny_id"] == "754085190"]
# df_wms_tf_sold_zero[df_wms_tf_sold_zero["tiny_sku"] == "7898571083767"]

,inventory_id,ml_code,seller_sku,title,tiny_id,tiny_sku,qtd_item,stock_replenishment,status,produtoCodigoInterno,produtoCodigoExterno,quantidade_disponivel
28,YAZB38863,MLB1440243016,FULLELIXIR3212,Encordoamento Elixir .012 Bronze 80/20 Nanoweb...,632364105,7897626832121,1,0.0,active,NaN,NaN,0


In [196]:
# # Identificar linhas duplicadas
# duplicated_rows = df_wms_tf_sold_zero[df_wms_tf_sold_zero.duplicated(subset=cols_to_check_duplicates, keep=False)]

In [197]:
# duplicated_rows[duplicated_rows['inventory_id']=='ELKB12099']
# duplicated_rows[duplicated_rows['tiny_id']=='754085190']
# duplicated_rows[duplicated_rows['tiny_sku']=='7898571083767']

In [198]:
# Agrupar por linhas repetidas e somar a coluna quantidade_disponivel
df_wms_tf_sold_zero_sum = (
    df_wms_tf_sold_zero.groupby(cols_to_check_duplicates)["quantidade_disponivel"]
    .sum()
    .reset_index()
)

# organizando
cols_result = [
    "inventory_id",
    "ml_code",
    "seller_sku",
    "title",
    "tiny_id",
    "tiny_sku",
    "qtd_item",
    "stock_replenishment",
    "status",
    "produtoCodigoInterno",
    "produtoCodigoExterno",
    "quantidade_disponivel",
]
df_wms_tf_sold_zero_sum = df_wms_tf_sold_zero_sum[cols_result]

In [199]:
df_wms_tf_sold_zero["quantidade_disponivel"] = (
    df_wms_tf_sold_zero["quantidade_disponivel"].fillna(0).astype("int64")
)

# Dados onde quantidade_disponivel = 0
df_wms_tf_sold_zero_less_zero = df_wms_tf_sold_zero[
    df_wms_tf_sold_zero["quantidade_disponivel"] < 1
]
df_wms_tf_sold_zero_less_zero = df_wms_tf_sold_zero_less_zero.drop_duplicates()

In [200]:
df_wms_tf_sold_zero = pd.concat(
    [df_wms_tf_sold_zero_sum, df_wms_tf_sold_zero_less_zero], ignore_index=True
)
df_wms_tf_sold_zero = df_wms_tf_sold_zero.drop_duplicates()

In [201]:
print(df_wms_tf_sold_zero.shape)
print(df_wms_tf_sold_zero_sum.shape)

(241, 12)
(148, 12)


In [202]:
# df_wms_tf_sold_zero_less_zero[df_wms_tf_sold_zero_less_zero['inventory_id']=='YAZB38863']

In [203]:
# df_wms_tf_sold_zero_sum[df_wms_tf_sold_zero_sum['inventory_id']=='YAZB38863']

In [204]:
# df_wms_tf_sold_zero_sum[df_wms_tf_sold_zero_sum["inventory_id"] == "ELKB12099"]
# df_wms_tf_sold_zero_sum[df_wms_tf_sold_zero_sum["tiny_id"] == "754085190"]
# df_wms_tf_sold_zero_sum[df_wms_tf_sold_zero_sum["tiny_sku"] == "7898571083767"]